In [4]:
pip install request

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement request (from versions: none)
ERROR: No matching distribution found for request


In [5]:
pip install beautifulSoup4

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1 in c:\users\ksy\anaconda3\lib\site-packages (from requests->webdriver-manager) (1.25.9)



In [8]:
pip install wordcloud

Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [2]:
import time
import re
import sys
from selenium import webdriver
from bs4 import BeautifulSoup
import csv
from urllib.request import urlopen
from selenium.webdriver import ActionChains

driver = webdriver.Chrome('C:/Users/KSY/Desktop/chromedriver.exe')
driver.get("https://www.idus.com/oc") #tab[0]
driver.execute_script('window.open("about:blank", "_blank");') #창 1개더 열어놓기 tab[1]
tabs = driver.window_handles
time.sleep(3)

def scroll():
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    while True:
        # 끝까지 스크롤 다운
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # 1초 대기
        time.sleep(1)

        # 스크롤 다운 후 스크롤 높이 다시 가져옴
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

with open ("ourdus_class.csv", 'w', encoding='utf-8',newline="") as fileWrite:
    myWriter = csv.writer(fileWrite)
    time.sleep(3)
    driver.switch_to_window(tabs[0])
    scroll()
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    
    myWriter.writerow(["카테고리", "이름","배너 사진 URL", "할인율", "가격/월",'전체가격','상세페이지 URL','수강시작일','수강기간','난이도','준비물','클래스소개','클래스댓글'])
       
    classlist = soup.findAll('li',{'class':'ui_grid__item'}) #모든 클래스 목록
    count = 0
    
    data=[] 
    
    #클래스 한개씩 작업
    for row in classlist:   
        #img_URL얻으려고 전처리과정
        img = row.find('div',{'class':'thumb-img'})
        img = img["style"]
        start=img.find('\"')
        end = img.find('\"',start+1)
        img_URL = img[start+1:end]

        #img_URL로 로컬 저장소에 사진 저장
        with urlopen(img_URL) as f:
            count +=1
            with open('./Ourdus/class_img/'+ str(count)+ '.jpg',"wb") as h:
                img=f.read()
                h.write(img)
        
        
        label = row.find('div',{'class':'label'}).text #카테고리
        name = row.find('div',{'class':'class-name'}).text #이름
        
        # 할인률 없는것들도 있어서 try~catch 사용
        try:
            discount = row.find('span',{'class':'sale-rate'}).text.replace('%','')
            discount=int(discount)
        except:
            discount=0
            
        #가격 전처리 (/월 가격임)
        price = row.find('span',{'class':'price'}).text.replace('\n','').replace(',','')
        price = int(price)
        
        data.append([label,name,img_URL,discount,price])
         
        # 현재 클래스의 상세 페이지 URL
        info_URL = row.find({'a':'data-v-27ea337a'})
        info_URL = info_URL["href"]
        info_URL = "https://www.idus.com/" + str(info_URL)
        
        
        driver.switch_to_window(tabs[1]) #두번째 탭으로 이동
        driver.get(info_URL)
        html = driver.page_source
        soup = BeautifulSoup(html, 'lxml')

        
        basic_info = soup.findAll({'dd':'data-v-af0d8610'}) #수강시작일 + 수강기간 + 난이도 + 준비물 다 긁어옴
        info=[]
        for i in basic_info: #한개씩 저장
            info.append(i.text)
        
        start_date=info[0] #수강시작일
        duration = info[1] #수강기간
        difficulty=info[2] #난이도
        material = info[3] #준비물
        
        #총 가격 전처리 (/월 말고)
        total_price = soup.find('p',{'class':'option-text price'}).text
        total_price=total_price.replace('원','').replace(',','')
        total_price=int(total_price)
        
        #클래스 정보 소개
        article = soup.findAll('p',{'class':'article_block__para bot_spacing'})
        article_info=[]
        for i in article:
            article_info.append(i.text)
        
        #클래스 댓글
        comment = soup.findAll('div',{'class':'area-txt'})
        comment_info=[]
        for i in comment:
            try:
                user_name=i.find({'span':'data-v-698f8bea'}).text
                user_comment=i.find('p',{'class':'txt'}).text
                comment_info.append([user_name,user_comment])   # [작성자 이름, 코멘트 내용] 이 한묶음
            except:
                continue

        tmp = data[count-1]+[total_price,info_URL,start_date,duration,difficulty,material,article_info,comment_info]
        
        myWriter.writerow(tmp)
        print(tmp)
        driver.switch_to_window(tabs[0])
    
 

SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 86
Current browser version is 88.0.4324.104 with binary path C:\Program Files (x86)\Google\Chrome\Application\chrome.exe


In [32]:
import time
import re
import sys
from selenium import webdriver
from bs4 import BeautifulSoup
import csv
from urllib.request import urlopen
from selenium.webdriver import ActionChains

driver = webdriver.Chrome('C:/Users/KSY/Desktop/chromedriver.exe')


 
tabs = driver.window_handles
driver.switch_to_window(tabs[0])
driver.get("https://www.idus.com/oc")

time.sleep(3)

html = driver.page_source
soup = BeautifulSoup(html, 'lxml')
classlist = soup.findAll('li',{'class':'ui_grid__item'})

for row in classlist:
        info_URL = row.find({'a':'data-v-27ea337a'})
        info_URL = info_URL["href"]
        info_URL = "https://www.idus.com/" + str(info_URL)
        print(info_URL)
        driver.switch_to_window(tabs[1])
        driver.get(info_URL)
        html = driver.page_source
        soup = BeautifulSoup(html, 'lxml')

        basic_info = soup.findAll({'dd':'data-v-af0d8610'})
        info=[]
        for i in basic_info:
            info.append(i.text)
        
        print(info)
        start_date=info[0]
        duration = info[1]
        difficulty=info[2]
        material = info[3]
        total_price = soup.find('p',{'class':'option-text price'}).text
        total_price=total_price.replace('원','')
        total_price=int(total_price)
        print(total_price)
        article = soup.findAll('p',{'class':'article_block__para bot_spacing'})
        article_info=[]
        for i in article:
            article_info.append(i.text)
        print(article_info)
        
        comment = soup.findAll('div',{'class':'area-txt'})
        comment_info=[]
        for i in comment:
            try:
                user_name=i.find({'span':'data-v-698f8bea'}).text
                user_comment=i.find('p',{'class':'txt'}).text
                comment_info.append([user_name,user_comment])
            except:
                continue
        print(comment_info)
            
        



<ipython-input-32-ef7a2cd331d4>:15: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(tabs[0])


https://www.idus.com//oc/class/140


IndexError: list index out of range

In [17]:
import time
import re
import sys
from selenium import webdriver
from bs4 import BeautifulSoup
import csv
from urllib.request import urlopen
from selenium.webdriver import ActionChains

driver = webdriver.Chrome('C:/Users/KSY/Desktop/chromedriver.exe')
driver.get("https://www.idus.com/oc") #tab[0]
driver.execute_script('window.open("about:blank", "_blank");') #창 1개더 열어놓기 tab[1]
tabs = driver.window_handles
time.sleep(3)

def scroll():
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    while True:
        # 끝까지 스크롤 다운
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # 1초 대기
        time.sleep(1)

        # 스크롤 다운 후 스크롤 높이 다시 가져옴
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

driver.switch_to_window(tabs[0])
scroll()
html = driver.page_source
soup = BeautifulSoup(html, 'lxml')
classlist = soup.findAll('li',{'class':'ui_grid__item'}) #모든 클래스 목록


data=[] #클래스 data 1개씩
# online_class_id , online_class_name, online_class_price, online_class_description, online_class_duration, online_class_level,
# online_class_start_date, preparation_flag, online_class_hit, online_class_purchase, online_class_like, online_class_rate,
#online_class_image, author_id,online_category_id

class_id = 0
author_id=0
category_id=0
category=[]
#클래스 한개씩 작업
for row in classlist:   
    
    author_id+=1
 
    #img_URL얻으려고 전처리과정
    img = row.find('div',{'class':'thumb-img'})
    img = img["style"]
    start=img.find('\"')
    end = img.find('\"',start+1)
    class_img = img[start+1:end]

    #img_URL로 로컬 저장소에 사진 저장
    with urlopen(class_img) as f:
        class_id +=1
        with open('C:/Users/KSY/Desktop/online_class_img/online_class_'+ str(class_id)+ '.jpg',"wb") as h:
            img=f.read()
            h.write(img)
        
    # 현재 클래스의 상세 페이지 URL
    info_URL = row.find({'a':'data-v-27ea337a'})
    info_URL = info_URL["href"]
    info_URL = "https://www.idus.com/" + str(info_URL)

    driver.switch_to_window(tabs[1]) #두번째 탭으로 이동
    driver.get(info_URL)
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml');
    
    category_name = soup.find('small',{"class":"sticky_aside__small"}).text
    class_name = soup.find('h2',{"class":"sticky_aside__title"}).text
    
    if category_name in category:
        category_id=category.index(category_name)+1
    else:
        category.append(category_name)
        category_id=len(category)
    
    class_video = soup.find('video',{"class":"fp-engine"})
    class_video = class_video["src"]
    
    basic_info = soup.findAll({'dd':'data-v-af0d8610'}) #수강시작일 + 수강기간 + 난이도 + 준비물 다 긁어옴
    
    start_date = basic_info[0].text
    class_duration = basic_info[1].text
    class_level = basic_info[2].text
    class_preparation = basic_info[3].text
    
     #총 가격 
    class_price = soup.find('p',{'class':'option-text price'}).text
    class_price=class_price.replace('원','').replace(',','')
    class_price=int(class_price)

    ##정보
    class_description = soup.find('p',{'class':'article_block__para bot_spacing'}).text
    
    # 즐겨찾기수
    class_like = soup.find('p',{'class':'oc-star-icon-count'}).text.strip()
    
    ##조회수, 별점,구매수
    class_hit=0
    class_rate=0
    class_purchase=0
    
    # online_class_id , online_category_id,author_id,online_class_name, online_class_price, online_class_duration, online_class_level,
    # online_class_start_date, preparation_flag, online_class_hit, online_class_purchase, online_class_like, online_class_rate,
    #online_class_image, class_video, online_class_description
    data.append([class_id,category_id,author_id,class_name,class_price,class_duration,class_level,start_date,class_preparation,class_hit,class_purchase,class_like,class_rate,class_img,class_video,class_description])
    print([class_id,category_id,author_id,class_name,class_price,class_duration,class_level,start_date,class_preparation,class_hit,class_purchase,class_like,class_rate,class_img,author_id,class_video,class_description])
  

<ipython-input-17-501ef276a8ed>:72: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(tabs[1]) #두번째 탭으로 이동


[1, 1, 1, '언제나 어디서나 함께, 나만의 양모 고양이 니들펠트 클래스', 159000, '180일', '쉬움', '신청 즉시', '구매가능', 0, 0, '459', 0, 'https://image.idus.com/image/files/7d992ccb52d142618ca885d0c435b5d2_720.jpg', 1, 'blob:https://www.idus.com/befc3c1d-56a9-4be5-a8ae-064c4b4c9c60', '안녕하세요~ \n고양이 만드는 고양이 집사, \n숨숨펠트입니다!\n\n아이디어스 클래스에서 \n만나 뵙게 되어 영광입니다.\n\n하루 종일 보고 또 봐도 보고 싶은 나의 고양이. \n학교에서도 직장에서도 어디서든 \n함께 하고 싶은 마음이 굴뚝같지 않나요?\n\n키링과 액자 제작으로 \n어디서든 함께 할 수 있도록 \n도와드릴게요.']
[2, 1, 2, '손쉽게 꼼지락 꼼지락, 재봉틀 입문하기', 119000, '180일', '쉬움', '3월 중순', '구매가능', 0, 0, '504', 0, 'https://image.idus.com/image/files/f89fddb344ea4f518566e50a1ff4b166_720.jpg', 2, 'blob:https://www.idus.com/48bf04e6-93ef-4100-923a-4855ecb43391', '📢 알려드려요!\n얼리버드 기간엔 가장 할인된 금액으로 구매 가능합니다.\n\n✔️ 얼리버드 기간 : 3월 12일(금) 오후 6시 마감\n✔️ 수업영상 오픈 : 3월 12일(금) 오후 6시\n✔️ 준비물은 3월 15일(월)부터 출고됩니다.\n\n📢 수요조사 쿠폰은 수강권과 준비물을 함께 구매할 때 사용할 수 있습니다. 아이폰 유저는 애플 정책으로 인해 앱이 아닌 웹에서 사용가능한 점을 꼭 참고해주세요!\n\n안녕하세요, 인디점입니다.\n재봉틀에 관심은 있지만\n어려울 거란 생각에 시작을 못했나요?\n아니면 재봉틀의 바늘이 무서운가요?😂\n\n항상 모든 것은 시작이 어려워

[19, 3, 19, '귀여움이 팡! 터지는 캐릭터 머랭 쿠키 베이킹', 119000, '180일', '쉬움', '3월 초', '구매가능', 0, 0, '668', 0, 'https://image.idus.com/image/files/d9f0cf4d76cb45618814c58c03a9cb96_720.png', 19, 'blob:https://www.idus.com/f76f42f0-7515-42e8-af0c-06a8e1f1e767', '📢 알려드려요!\n얼리버드 기간엔 가장 할인된 금액으로 구매 가능합니다.\n\n✔️ 얼리버드 기간 : 3월 9일 오후 6시 마감\n✔️ 수업영상 오픈 : 3월 9일 오후 6시\n✔️ 준비물은 3월 10일부터 출고됩니다.\n\n📢 수요조사 쿠폰은 수강권과 준비물을 함께 구매할 때 사용할 수 있습니다. 아이폰 유저는 애플 정책으로 인해 앱이 아닌 웹에서 사용가능한 점을 꼭 참고해주세요!\n\n안녕하세요.\n귀여운 머랭 쿠키를 만드는\n쥬쥬# 디저트입니다.\n\n이번 아이디어스 클래스에서는\n기본 머랭 쿠키부터 시작하여\n귀여움이 터지는 캐릭터 머랭 쿠키까지\n총 6종류의 머랭 쿠키를 알려드릴게요.\n\n오븐만 있다면 누구나 쉽게!\n도전해 보실 수 있는 클래스입니다.😊']
[20, 12, 20, '아이패드로 그리는 보송보송 일러스트', 149000, '180일', '쉬움', '신청 즉시', '없음', 0, 0, '18,055', 0, 'https://image.idus.com/image/files/8482cdb67f8d47d284c9afffeae53df0_720.png', 20, 'blob:https://www.idus.com/1e32db0f-24a4-43cd-91e5-ec2ca96b0d25', '✔️ 준비물은 2월 16일 후, 이메일로 발송됩니다.\n      작가 문의 메시지로 이메일 주소를 보내주세요. \n\n📢 아이패드와 애플 펜슬, 프로크리에이트 앱을 개인 준비물로 갖추셔야 하는 클래스입니다.']
[21, 8, 21, '초보

[39, 3, 39, '유튜브 100만 조회로 인정 받은 맛! 에그타르트 인생 레시피', 52000, '180일', '쉬움', '신청 즉시', '구매가능', 0, 0, '2,144', 0, 'https://image.idus.com/image/files/44c9c2e5d35b400e901681cee3ae1a1f_720.jpg', 39, 'blob:https://www.idus.com/49769c30-85cd-4243-8e4e-e3fc36af0c59', '안녕하세요 :)\n베이킹 유튜브 크리에이터\n그루밍식당입니다😉\n\n여러분은 에그타르트 좋아하시나요?\n카페에서 사먹던 바로 그 에그타르트! \n집에서도 손쉽게 만들어 먹을 수 있다면\n얼마나 좋을까요😭']
[40, 12, 40, '아이패드로 그리는, 살아 움직이는 디지털 색연필', 159000, '180일', '쉬움', '신청 즉시', '없음', 0, 0, '21,784', 0, 'https://image.idus.com/image/files/b2834527af3346aa86066ad4f181c061_720.jpg', 40, 'blob:https://www.idus.com/54203b2d-09cd-41cb-835a-8cd495f26d8f', '📢 아이패드와 애플펜슬을 개인 준비물로\n갖추셔야 하는 클래스입니다.\n\n안녕하세요.\n색연필로 따뜻한 일러스트를 그리는\n원스입니다✨']
[41, 3, 41, '새하얀 눈 위에 보석으로 물든 찹쌀떡, 셋빼 화과자', 79000, '180일', '쉬움', '신청 즉시', '구매가능', 0, 0, '2,001', 0, 'https://image.idus.com/image/files/97455682907242ec8448cd0dc0ad8452_720.jpg', 41, 'blob:https://www.idus.com/0b025931-065e-403a-8a80-bcebe716a5b5', '✔️본 클래스는 무제한 수강권 제외 클래스입니다.\n✔️ 준비물 키트 배송은 3-5일 소요

[61, 21, 61, '지그펜 캘리그래피와, 플러스펜으로 꾸미는 수채 그림', 129000, '180일', '쉬움', '신청 즉시', '구매가능', 0, 0, '1,862', 0, 'https://image.idus.com/image/files/cc61b557d3744e79964eb3eb6f543fa1_720.jpg', 61, 'blob:https://www.idus.com/68d3e79c-8b27-4183-9b75-34ddc89045b9', '안녕하세요. 그린나래캘리입니다.\n\n오랫동안 북&편집 디자이너로 활동하며\n나만의 글씨를 꿈꾸다 자연스레\n캘리그래피를 만나게 되었고,\n\n지금은 ‘그린나래 작업실’를 운영하며\n캘리그래피와 그림 작업을 하고 있습니다.']
[62, 3, 62, '달달한 꽃을 피워내는, 앙금 플라워 쿠키 클래스', 69000, '180일', '쉬움', '신청 즉시', '구매가능', 0, 0, '2,076', 0, 'https://image.idus.com/image/files/3fda2bea3a704438adfb9ede8b77176c_720.jpg', 62, 'blob:https://www.idus.com/966c46f6-7a8a-4150-86a5-7a1dcead52d6', '안녕하세요 오직아뜰리에 입니다 🌺\n플라워 케이크와 크래프트 작품을 만드는 \n공방을 운영하고 있으며 \n현재 유튜브 크리에이터로도 활동하고 있습니다.']
[63, 8, 63, '귀여운 히팝 몬스터 키링&스마트톡 만들기', 40000, '180일', '쉬움', '신청 즉시', '구매가능', 0, 0, '1,779', 0, 'https://image.idus.com/image/files/aede12932e614b7f9a3ea1937b9bfae8_720.jpg', 63, 'blob:https://www.idus.com/d3bc256b-3a5d-4a8a-8f84-f4fedc5763a6', '안녕하세요. \n귀여움을 찾아 함께 떠나는 \n뽀실이탐험대입니다!\n\n다가

[83, 23, 83, '천년의 빛을 조각하는 자개공예 클래스', 169000, '180일', '쉬움', '신청 즉시', '구매가능', 0, 0, '1,646', 0, 'https://image.idus.com/image/files/d7111f820c8c4be3b9cd5bb5f855a781_720.jpg', 83, 'blob:https://www.idus.com/a8326adf-cf9d-44f3-bf94-cb54ac96a64e', '안녕하세요 50여 년간 자개 공예의 길을 걸어온\n진주쉘 이영옥 명장입니다.\n\n자개공예의 전통을 널리 알리기 위해\n이번 클래스를 준비하게 되었습니다.']
[84, 16, 84, '깎을수록 뚜렷해지는 나의 취향, 목조각 클래스', 199000, '180일', '쉬움', '신청 즉시', '구매가능', 0, 0, '2,630', 0, 'https://image.idus.com/image/files/0e7de6aca0a64087bc3fe667853aaa8f_720.jpg', 84, 'blob:https://www.idus.com/4420a31e-7b62-462d-8f6b-33a5fed05b3d', '안녕하세요.\n목조각 작가 윤소라입니다.\n\n나무를 이용해 일상생활 속\n즐거움을 주는 다양한 생활 소품을 만드는\n목조각 공방 ‘스튜디오 앤캣’을 운영하고 있습니다😸']
[85, 4, 85, '시원함을 엮는 매듭 공예 마크라메 네트백 클래스', 30000, '180일', '쉬움', '신청 즉시', '구매가능', 0, 0, '809', 0, 'https://image.idus.com/image/files/8048b8eed7c049bf8332fc20a4755376_720.jpg', 85, 'blob:https://www.idus.com/fe4e92a8-dde4-4ade-bca9-5ec1f8862ad5', '안녕하세요 두 손으로 실을 엮어가는\n손재주공장 함초롬입니다. 🙌\n\n여러가지 아이템을 만드는 게\n부담스러운 분들을 위해\n\n하루

[104, 13, 104, '일상을 꽃으로 채워보세요. 플로리스트 입문하기', 149000, '180일', '쉬움', '신청 즉시', '구매가능', 0, 0, '1,496', 0, 'https://image.idus.com/image/files/95c2f350941d40d6a71ea4d8f724f46a_720.jpg', 104, 'blob:https://www.idus.com/ffc0ae76-0efe-4d9d-af77-7551b653e563', '안녕하세요.\n플라워공방 아뜰리에 라모 입니다.\n꽃이 주는 힘을 느껴보신 적 있으신가요?\n싱그러운 생화에서만 느낄 수 있는 에너지를 전달해 드릴게요.']
[105, 11, 105, '감성저격 물성형 입체 가죽공예', 169000, '180일', '쉬움', '신청 즉시', '구매가능', 0, 0, '1,328', 0, 'https://image.idus.com/image/files/de363b1b2fce46d4a20af21cd084ebb6_720.jpg', 105, 'blob:https://www.idus.com/d6132edb-2f67-45de-8612-b126425f9fe2', '안녕하세요, 가죽공예 크리에이터 하안입니다.\n\n가죽공예로 전세계 구독자분들과 소통하기 시작한 이후로, 강의나 키트에 관한 많은 문의를 받았었습니다.\n그래서 이번 금손클래스를 통해 많은 분들이 가장 궁금해하셨던 수업을 준비했습니다.\n\n어려워보이신다구요?\n공방 옆자리 친구처럼 과정을 하나하나 자세히 알려드려 처음해보시더라도 충분히 가능하십니다.\n그리고 더 쉽게 작업을 하실 수 있도록 저만의 팁과 아이디어를 모두 담았습니다. 제가 옆에서 도와드릴게요!\n\n초보도 쉽게 따라할 수 있는 가죽공예, 같이 시작하실까요?']
[106, 18, 106, '내 피부에 꼭 맞춘, 홈메이드 수제비누 클래스', 149000, '180일', '쉬움', '신청 즉시', '구매가능', 0, 0, '1,365', 0, 'https://image.idus

[125, 8, 125, '한 줄 한 줄 실과 함께 취향을 엮는 위빙 공예', 199000, '180일', '쉬움', '신청 즉시', '구매가능', 0, 0, '3,474', 0, 'https://image.idus.com/image/files/bf696eab1d4c4d648eb4de60c8a09ff7_720.jpg', 125, 'blob:https://www.idus.com/64da7fad-b74b-4427-a732-ae0c0a51d375', '안녕하세요.\n위빙 공방 하라두의 작가 원하라입니다.\n\n🧵위빙을 아시나요?\n     위빙이란, 가로와 세로로 실이 교차하여\n     섬유를 짜는 공예입니다.']
[126, 2, 126, '반짝반짝 투명한 매력, 누구나 쉽게 배우는 레진공예', 105000, '180일', '쉬움', '신청 즉시', '구매가능', 0, 0, '3,054', 0, 'https://image.idus.com/image/files/4b26df21576b45b990969b5439e7cc49_720.jpg', 126, 'blob:https://www.idus.com/f6f25046-b22e-4653-8bbe-2d4b5ae2c2a8', '안녕하세요👋🏻\n레진공예 유튜브채널 ‘마니랜드’를\n운영하고 있는 마니라고 합니다.\n\n반짝이고 만드는 것을 좋아하는 분들에게\n최고의 취미가 되어줄 레진공예를 소개합니다 :)']
[127, 24, 127, '자연식물식으로 내 속을 편안하게, 로푸드 레시피', 149000, '180일', '쉬움', '신청 즉시', '구매가능', 0, 0, '1,471', 0, 'https://image.idus.com/image/files/75cce2bc3c434806b9d1fbbcfb8eb77c_720.jpg', 127, 'blob:https://www.idus.com/1962a9bd-8f16-49b7-bedb-08f7bc0498f9', "안녕하세요! 비건 로푸드(Raw Food) 레시피를 연구·개발하는 리틀엔팬트리입니

In [35]:
with open ("ourdus_online_class.csv", 'w', encoding='utf-8',newline="") as fileWrite:
    myWriter = csv.writer(fileWrite)

    myWriter.writerow(["online_class_id" , "online_category_id","author_id","online_class_name", "online_class_price", 
                       "online_class_duration", "online_class_level","online_class_start_date", "preparation_flag", 
                       "online_class_hit", "online_class_purchase", "online_class_like", "online_class_rate",
                       "online_class_image","class_video", "online_class_description"])
    
    for one in data:
        #Preparation Flag 전처리
        if one[8]=="없음":
            one[8]=False
        else:
            one[8]=True
            
        #수강날짜일 전처리
        if(type(one[5])!=int):
            one[5]=one[5].replace("일","")
            one[5]=int(one[5])
        if(type(one[11])!=int):
            one[11]=one[11].replace(",","")
            one[11]=int(one[11])

        myWriter.writerow(one)

In [40]:
with open ("ourdus_online_class_category.csv", 'w', encoding='utf-8',newline="") as fileWrite:
    myWriter = csv.writer(fileWrite)
    myWriter.writerow(["online_category_id","online_category_name"])
    cid=0
    for i in category:
        cid+=1
        myWriter.writerow([cid,i])
        
        
        

AttributeError: 'str' object has no attribute 'text'

In [37]:
print(category)

['바느질/재봉', '레진', '디저트/음료', '위빙/마크라메', '색연필/크레용', '유화/아크릴화', '페이퍼아트/포장', '뜨개/코바늘', '뷰티', '펜 드로잉', '가죽', '디지털 드로잉', '플라워/가드닝', '유리', '수채화', '우드/라탄', '점토', '비누', '도자', '캔들/발향', '캘리그래피/서예', '금속', '자개', '쿠킹', '실크스크린/판화']
